In [ ]:
# Python 3.11
# Harry Mancinelli
# Play Blackjack

from card_deck import *
import numpy as np
import pandas as pd
import copy

In [ ]:
deck = create_deck()
play_token = False
dealer = []
player = []
d_score = 0
p_score = 0
ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'Jack', 'Queen', 'King',"Ace"]

In [ ]:
def draw_card() :
    card_index = np.random.randint(0,len(deck))
    take_card = deck[card_index]
    deck.remove(take_card)
    return take_card


def display_card(card) :
    print(card[0], "of", card[1])
    

def calculate_score(card) :
    temp = card[0]
    index = ranks.index(temp) + 2
    if index in [11,12,13] :
        index = 10
    elif index == 14 :
        index = 11
    return index

def hard_reset() :
    global deck, dealer, player, d_score, p_score, play_token
    dealer = []
    player = []
    d_score = 0
    p_score = 0
    play_token = False
    
    if len(deck) < np.random.randint(60,90) :
        deck = 6*create_deck() 
        print("\n\nNew Deck")

In [ ]:
# Currency

currency = 100

## Rules
- No splitting
- Blackjack pays 3-2
- Soft aces for dealer and player
- Dealer stands on soft 17
- 6 Decks in use 
- Decks are cut when there are between 60 and 90 cards left

In [ ]:
def play_blackjack(action = "stand", bet = 1) :
    
    if 'deck' not in globals() :
        deck = 6*create_deck()
    
    global play_token
    global dealer
    global player
    global p_score
    global d_score
    global currency
    
    # Deal Hands
    
    if not play_token :
        
        global hidden_bet
        hidden_bet = copy.deepcopy(bet)
        
        global ace_count
        ace_count = 0
        
        if hidden_bet == 0 :
            print("Place a bet brokie!!")
            return
        
        if currency < hidden_bet :
            print("Current Balance:", currency,"Is Too Small To Fund This Bet \nYou're Broke!!!")
            return
        
        currency = currency - hidden_bet
        
        player.append(draw_card())
        dealer.append(draw_card())
        player.append(draw_card())
        dealer.append(draw_card())
        
        play_token = True
        
        #Print Bet and Currency
        print("Bank",currency,"\nCurrent Bet:",bet,"\n")
        
        # Dealer Hand
        print("Dealer's Hand:")
        display_card(dealer[0])  
        print("Down Card")
        
        # Dealer Score
        d_score = calculate_score(dealer[0])
        print("\nDealer's Score:",d_score)
        
        # Player Hand
        print("\nYour Hand:")
        for card in player :
            display_card(card)
            
        # Player Score
        for card in player :
                p_score += (calculate_score(card)) 
        print("\nYour Score:", p_score)
        
        result = np.logical_and(np.isin(player[0][0], np.array(['10','Jack','Queen','King'])) or
                        np.isin(player[1][0], np.array(['10','Jack','Queen','King'])),
                        'Ace' in [player[0][0],player[1][0]])
        
        if result :
            print("\nBLACKJACK!!!! You Win")
            
            currency = currency + (2.5*hidden_bet)
            
            hard_reset()              
        
        
        return
        
    # Hit or Stand   
    if play_token :
        
        # Hit
        if action == "hit" :
            player.append(draw_card())
            
            # Print Sequence
            #Print Bet and Currency
            print("Bank",currency,"\nCurrent Bet:",bet,"\n")
            
            # Dealer Hand
            print("Dealer's Hand:")
            display_card(dealer[0])  
            print("Down Card")
            
            # Dealer Score
            d_score = calculate_score(dealer[0])
            print("\nDealer's Score:",d_score)
            
            # Player Hand
            print("\nYour Hand:")
            for card in player :
                display_card(card)
                
            # Player Score
            p_score += (calculate_score(player[-1])) 
            
            if p_score <= 21 :
                print("\nYour Score:", p_score)
            
            elif p_score > 21 :
                
                aces = sum(card[0] == 'Ace' for card in player)
                
                if aces == 0 : 
                
                    print("\nYour Score:", p_score)
                    print("\nBUST")
                    hard_reset()
                    
                if aces > 0 :
                    
                    ace_count = ace_count + 1
                    
                    if ace_count > aces :
                        print("\nYour Score:", p_score)
                        print("\nBUST")
                        hard_reset()
                        
                    else :                    
                        
                        p_score = p_score - 10
                        print("\nYour Score:", p_score)
                    
                    
                
                
            ## Stand
        if action == "stand" :
            
            d_score += (calculate_score(dealer[1]))
            
            global dealer_aces_count
            dealer_aces_count = 0
            
            while d_score < 17 and d_score <= p_score :
                dealer.append(draw_card())
                d_score += (calculate_score(dealer[-1]))
                
                dealer_aces = sum(card[0] == 'Ace' for card in dealer)
                
                if d_score > 21 and dealer_aces > 0 :
                    dealer_aces_count = dealer_aces_count + 1
                    
                    if dealer_aces >= dealer_aces_count :
                        d_score = d_score - 10
                
            ## Print Sequence
            #Print Bet and Currency
            print("Bank",currency,"\nCurrent Bet:",bet,"\n")   
            
            # Dealer Hand
            print("Dealer's Hand:")
            for card in dealer :
                display_card(card)
            
            # Dealer Score
            print("\nDealer's Score:",d_score)
            
            # Player Hand
            print("\nYour Hand:")
            for card in player :
                display_card(card)
                
            # Player Score
            print("\nYour Score:", p_score)
            
                
            if d_score > 21 or p_score > d_score :
                print ("You Win", hidden_bet)
                hard_reset()
                
                currency = currency + (2*hidden_bet)
            
            elif d_score == p_score :
                print("Draw")
                hard_reset()
                
                currency = currency + (hidden_bet)
                
            else :
                print("You Lose")
                hard_reset()
                



In [ ]:
play_blackjack(action = "hit", bet = 1)